In [1]:
!pip install jieba

In [2]:
!pip install gensim

In [3]:
!mkdir jieba_data

mkdir: cannot create directory ‘jieba_data’: File exists


In [4]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O jieba_data/dict.txt.big

--2020-06-01 18:26:59--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2020-06-01 18:26:59--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘jieba_data/dict.txt.big’

jieba_data/dict.txt 100%[===================>]   8.18M  --.-KB/s    in 0.07s   

2020-06-01 18:26:59 (117 MB/s) - ‘jieba_data/dict.txt.big’ saved [8583143/8583143]



### 載入 jieba 函式庫

In [5]:
import jieba
import time

### 載入繁體中文詞庫

In [6]:
jieba.set_dictionary('jieba_data/dict.txt.big')

### 開啟文字檔

In [7]:
# 開檔
fileAllLines = []
with open('Three Kingdoms.txt') as fileLine:
    for line in fileLine:
        #print(line)
        fileAllLines.append(line)
    

In [8]:
fileAllLines

['《三國演義》作者：羅貫中\n',
 '\n',
 '簡介\n',
 '\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。\n',
 '\u3000\u3000羅貫中（1330年一1400年之間），名本，號湖海散人，明代通俗小說家。他的籍貫一說是太原（今山西），一說是錢塘（今浙江杭州），不可確考。據傳說，羅貫中曾充任過元末農民起義軍張士誠的幕客．除《三國誌通俗演義》外，他還創作有《隋唐志傳》等通俗小說和《趙太祖龍虎風雲會》等戲劇。另外，有相當一部分人認為《水滸傳》後三十回也是其所作。\n',
 '\n',
 '目錄\n',
 '\n',
 '第001回\u3000宴桃園豪傑三結義\u3000斬黃巾英雄首立功 第002回\u3000張翼德怒鞭督郵\u3000何國舅謀誅宦豎 \n',
 '第003回\u3000議溫明董卓叱丁原\u3000饋金珠李肅說呂布 第004回\u3000廢漢帝陳留踐位\u3000謀董賊孟德獻刀 \n',
 '第005回\u3000發矯詔諸鎮應曹公\u3000破關兵三英戰呂布 第006回\u3000焚金闕董卓行兇\u3000匿玉璽孫堅背約 \n',
 '第007回\u3000袁紹磐河戰公孫\u3000孫堅跨江擊劉表 第008回\u3000王司徒巧使連環計\u3000董太師大鬧鳳儀亭 \n',
 '第009回\u3000除暴凶呂布助司徒\u3000犯長安李傕聽賈詡 第010回\u3000勤王室馬騰舉義\u3000報父仇曹操興師 \n',
 '第011回\u3000劉皇叔北海救孔融\u3000呂溫侯濮陽破曹操 第012回\u3000陶恭祖三讓徐州\u3000曹孟穗大戰呂布 \n',
 '第013回\u3000李傕郭汜大交兵\u3000楊奉董承雙救駕 第014回\u3000曹孟德移駕幸許都\u3000呂奉先乘夜襲徐郡 \n',
 '第015回\u3000太史慈酣鬥小霸王\u3000孫伯符大戰嚴白虎 第016回\u3000呂奉先射戟轅門\u3000曹孟德敗師淯水 \n',
 '第017回\u3000袁公路大起七軍\u3000曹孟德會合三將 第018

### 斷詞測試

#### 詞性說明: https://gist.github.com/luw2007/6016931

In [9]:
import jieba.posseg as pseg
jieba.enable_paddle()
text = "三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。"
results = pseg.lcut(text, use_paddle=True)
s = []
for w, p in results:
    if p in ['n', 'nr','ns','nt','nz','nl','ng', 't', 's','vn']:
        print("%s, %s"%(w, p))
        s.append(w)
new_line = ' '.join(s)
new_line

Paddle enabled successfully......


三國演義, nz
歷史, n
小說，, nr
古代, t
篇章, n
作，, vn
名著, n
。, n
作者, n
羅貫, n
故事, n
黃巾, n
義, n
，終, nr
。, n
書陳, nr
，, n
萬事，七實三虛。三國, nz
蜀，吳。小說通篇精巧敘述謀略，, nt
中國謀略全書。, nz


'三國演義 歷史 小說， 古代 篇章 作， 名著 。 作者 羅貫 故事 黃巾 義 ，終 。 書陳 ， 萬事，七實三虛。三國 蜀，吳。小說通篇精巧敘述謀略， 中國謀略全書。'

### 進行斷字斷詞

In [10]:
# 斷詞
start_time = time.time()
import jieba.posseg as pseg
#jieba.enable_paddle()
jieba.enable_parallel(4)
seg = []
for i in range(len(fileAllLines)):
    #cut_list = list( jieba.cut(fileAllLines[i], cut_all = False) )
    #seg.append([' '.join([ item for item in cut_list if len(item) > 1 ])])
    cut_result = list(pseg.cut(fileAllLines[i], use_paddle=False))
    each_line_list = []
    for w, p in cut_result:
        #print("%s: %s"%(w, p))
        if p in ['n', 'nr','ns','nt','nz','nl','ng', 't', 's','vn']:
            #print("%s: %s"%(w, p))
            each_line_list.append(w)
    new_line = ' '.join(each_line_list)
    #print(new_line)
    seg.append(new_line)
print("--- spend %s seconds ---" % (time.time() - start_time))

Building prefix dict from /home/jovyan/work/jieba_data/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ud3c40470d3518910bfd0dc86d865fad4.cache
Loading model cost 2.033 seconds.
Prefix dict has been built successfully.


--- spend 535.223025560379 seconds ---


In [11]:
print(len(seg))

6001


In [12]:
#seg = [ s for s in seg if len(s) > 0] 
seg

['三國演義 作者 羅貫中',
 '',
 '',
 '三國演義 中國 古代 章回小說 開山 作 作者 明朝 羅貫中 故事 黃巾起義 西晉 統一 書 陳敘 萬事 三國 魏 吳 小說 通篇 謀略 中國 謀略 全書',
 '羅貫中 名本 湖海 散人 明代 籍貫 太原 山西 錢塘 浙江 杭州 據傳說 羅貫中 過元末 農民 起義軍 張士誠 幕客 三國 通俗 演義 創作 隋唐 志傳 太祖 龍虎風雲 戲劇 人 水滸傳 作',
 '',
 '目錄',
 '',
 '宴 桃園 結義 黃巾 英雄 立功 張翼德 鞭督 郵 何國舅 宦',
 '議溫 董卓 饋金 珠 李肅 呂布 漢帝 陳留 董賊 孟德獻刀',
 '矯詔 諸鎮 曹公 兵 戰呂布 金闕 董卓 行兇 玉璽 孫堅 背約',
 '袁紹 磐河 戰 公孫 孫堅 江 劉表 王司徒 董 太師 鳳儀亭',
 '除暴 呂布 司徒 長安 李 賈 王室 馬騰 報父仇 曹操 興師',
 '劉皇叔 北海 孔融 呂溫侯 濮陽 曹操 陶恭祖 徐州 曹孟穗 戰呂布',
 '李 郭 交兵 楊奉 董承雙 曹孟德 呂奉先 徐郡',
 '太史慈 小霸王 孫伯符 大戰 嚴白虎 呂奉先 射戟 轅門 曹孟德 師 水',
 '袁 公路 軍 曹孟德 賈文 料 敵 夏侯惇 矢 睛',
 '邳城 曹操 白門樓 呂布 曹阿 許田 董國舅 內閣 詔',
 '曹操 酒論 英雄 關公 城 車胄 袁 曹 關張 擒王 劉二',
 '衣 賊 吉 太醫 國賊 行兇 貴妃 皇叔 投 袁紹',
 '屯 土山 關公 事 白馬 曹操 重圍 袁本初 敗兵 關雲長 封金',
 '美髯公 走單騎 漢壽 侯 五關 蔡陽 兄弟 古城 主臣 聚義',
 '小霸王 於吉 碧 眼兒 江東 戰 官渡 本初 敗績 烏巢 孟德 糧',
 '曹操 倉 亭 本初 玄德 荊州 劉表 冀州 袁尚 漳河',
 '曹丕 甄氏 郭嘉遺 遼東 蔡夫人 屏 密語 劉皇叔 過檀溪',
 '玄德 南漳 單福 新野 英主 玄德 樊城 走馬 諸葛',
 '司馬徽 名士 劉玄德 隆中 決策 戰 長江 孫氏',
 '荊州 城 公子 博望坡 軍師 用兵 蔡夫人 議獻 荊州 諸葛亮 火燒 新野',
 '劉玄德 攜民 渡江 趙子龍 單騎 張翼德 長阪 橋 劉豫州 漢津口',
 '諸葛亮 舌戰 群儒 魯子敬 眾議 孔明 周瑜 孫權 曹操',
 '三江口 曹操 折

### 準備存檔，將斷好的字詞存下來。先刪除之前留下的紀錄檔案(segDone.txt)

In [13]:
!rm -f segDone.txt

### 斷詞結果存檔

In [14]:
# 斷詞結果存檔
segSaveFile = 'segDone.txt'
with open(segSaveFile, 'wb') as saveFile:
    for i in range(len(seg)):
        #words = seg[i][0].encode('utf-8')
        words = seg[i].encode('utf-8')
        if len(words) > 0:
            saveFile.write(words)
            saveFile.write('\n'.encode())

In [15]:
!head -10 segDone.txt

三國演義 作者 羅貫中
三國演義 中國 古代 章回小說 開山 作 作者 明朝 羅貫中 故事 黃巾起義 西晉 統一 書 陳敘 萬事 三國 魏 吳 小說 通篇 謀略 中國 謀略 全書
羅貫中 名本 湖海 散人 明代 籍貫 太原 山西 錢塘 浙江 杭州 據傳說 羅貫中 過元末 農民 起義軍 張士誠 幕客 三國 通俗 演義 創作 隋唐 志傳 太祖 龍虎風雲 戲劇 人 水滸傳 作
目錄
宴 桃園 結義 黃巾 英雄 立功 張翼德 鞭督 郵 何國舅 宦
議溫 董卓 饋金 珠 李肅 呂布 漢帝 陳留 董賊 孟德獻刀
矯詔 諸鎮 曹公 兵 戰呂布 金闕 董卓 行兇 玉璽 孫堅 背約
袁紹 磐河 戰 公孫 孫堅 江 劉表 王司徒 董 太師 鳳儀亭
除暴 呂布 司徒 長安 李 賈 王室 馬騰 報父仇 曹操 興師
劉皇叔 北海 孔融 呂溫侯 濮陽 曹操 陶恭祖 徐州 曹孟穗 戰呂布


In [16]:
!cat segDone.txt | wc -c

751665


### 載入word2vec 函式庫
### 參考網頁：https://radimrehurek.com/gensim/models/word2vec.html

In [17]:
from gensim.models import word2vec

### 一行一行的從檔案中取出句子：一行一個句子，句子中的字必須預先處理好並以空白隔開

In [18]:
sentences = word2vec.LineSentence("segDone.txt")

### 訓練模型
#### size: 向量維度 = 300
#### sg: 0(CBOW), 1(Skip-gram)

In [19]:
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=300, sg=1, window=10, workers=3, min_count=2)
print("--- spend %s seconds ---" % (time.time() - start_time))

--- spend 6.451516151428223 seconds ---


In [20]:
model

### 儲存訓練好的word2vec 模型

In [21]:
model.save("word2vec.model")

In [22]:
model.corpus_total_words

114600

# 詞相似度

In [23]:
model.wv.most_similar('赤壁')

[('咨', 0.996192455291748),
 ('唐', 0.9960330128669739),
 ('張昭奏', 0.9946820139884949),
 ('凌操', 0.9943174123764038),
 ('黃祖部', 0.9931323528289795),
 ('土', 0.992579460144043),
 ('孤', 0.9925642013549805),
 ('官授', 0.9925411343574524),
 ('吳郡', 0.9924865961074829),
 ('妹嫁', 0.9923616051673889)]

In [24]:
model.wv.similar_by_word('赤壁')

[('咨', 0.996192455291748),
 ('唐', 0.9960330128669739),
 ('張昭奏', 0.9946820139884949),
 ('凌操', 0.9943174123764038),
 ('黃祖部', 0.9931323528289795),
 ('土', 0.992579460144043),
 ('孤', 0.9925642013549805),
 ('官授', 0.9925411343574524),
 ('吳郡', 0.9924865961074829),
 ('妹嫁', 0.9923616051673889)]

In [25]:
model.wv.similarity('曹操','呂布')

0.4397983

In [26]:
model.wv.similar_by_word('曹操')

[('張魯', 0.9287075996398926),
 ('郭圖', 0.9183179140090942),
 ('冀州', 0.9121900200843811),
 ('樊城', 0.904861569404602),
 ('商議', 0.9010980129241943),
 ('空虛', 0.8989623785018921),
 ('袁紹', 0.8937135934829712),
 ('許都', 0.8926887512207031),
 ('星夜', 0.8926668167114258),
 ('官渡', 0.8925915360450745)]

In [27]:
model.wv.similar_by_word('呂布')

[('布', 0.9009829759597778),
 ('陳宮', 0.8335521221160889),
 ('徐州', 0.7477862238883972),
 ('兗州', 0.7375565767288208),
 ('袁術', 0.7300129532814026),
 ('濮陽', 0.7190690636634827),
 ('魏續', 0.6730461716651917),
 ('呂', 0.6707903146743774),
 ('紀靈', 0.6704277992248535),
 ('宋憲', 0.662251353263855)]

In [28]:
model.wv.similarity('孔明','諸葛亮')

0.8270606

In [29]:
model.wv.similar_by_word('孔明')

[('孔明曰', 0.8840631246566772),
 ('孔明笑', 0.8693668246269226),
 ('鄧芝', 0.8609883785247803),
 ('孟獲', 0.8593840003013611),
 ('馬謖', 0.8570560216903687),
 ('諸葛亮', 0.8270605802536011),
 ('洞', 0.8192802667617798),
 ('計策', 0.817386269569397),
 ('楊儀', 0.8101282119750977),
 ('雍', 0.8086900115013123)]

In [30]:
model.wv.similar_by_word('諸葛亮')

[('計', 0.9555791616439819),
 ('雍', 0.9510435461997986),
 ('南郡', 0.9492337703704834),
 ('渡江', 0.9400891661643982),
 ('領命', 0.9327317476272583),
 ('用兵', 0.9262199997901917),
 ('孔明之', 0.9239003658294678),
 ('計策', 0.921851634979248),
 ('人報', 0.9180041551589966),
 ('心中', 0.9167276620864868)]

In [31]:
model.wv.similarity('孔明','周瑜')

0.6405705

In [32]:
model.wv.similarity('孔明','司馬懿')

0.7805211

In [33]:
#input_string = input('關鍵字:')
#model.wv.similar_by_word(input_string)

# 詞預測

In [34]:
result = model.predict_output_word('關羽')
result

[('漢中', 0.00045638563),
 ('司馬懿', 0.0004247168),
 ('魏延', 0.00041244522),
 ('魏', 0.00040726128),
 ('黃忠', 0.00040084688),
 ('汝', 0.00037348224),
 ('魏兵', 0.00036071264),
 ('忠', 0.00035421096),
 ('王', 0.00033988425),
 ('人馬', 0.00031910455)]

# 類推 (analogy)

#### 基本範例 (https://radimrehurek.com/gensim/models/keyedvectors.html)

In [35]:
import gensim.downloader as api

In [36]:
word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data

In [37]:
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [38]:
result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.8965


In [39]:
ans1 = model.wv.most_similar(positive=['曹操','孔明'], negative=['孫權'])
print("%s: %s"%(ans1[0]))

新野: 0.8498674035072327


In [40]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

In [41]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

In [42]:
print(x_vals)
print(y_vals)
print(labels)

[-101.38119, -97.99631, -76.4459, 38.854206, 10.427899, 19.169365, 55.664978, -81.77365, -72.378136, 63.78302, -16.73654, 21.864346, 45.895824, 74.029945, 35.93778, -89.9458, -78.319756, 39.30046, -34.066093, -71.139885, 46.569687, 42.176205, 63.8508, 64.10836, 63.912964, 5.4988484, 67.556595, 39.1072, 50.053352, -32.007515, -27.719025, -79.989296, -24.572157, -102.02899, 70.84775, -100.41475, -63.436596, 42.953033, 63.668743, 20.81743, -5.306026, 31.326359, -101.86196, 24.069046, -101.89267, 66.1581, 64.545525, -12.984455, -99.0808, -10.5474205, 72.550735, 36.399284, -99.18905, 57.216415, -44.153435, 72.30152, 36.160152, 33.19489, 57.357468, 27.198565, 70.99398, 69.46647, 32.255672, -65.91624, 70.14872, 46.524864, 43.70821, -77.64993, 61.24851, 1.0199262, 22.361437, 55.950504, 60.63963, 64.57701, -5.9346414, 4.6300163, -87.54178, 33.709427, -26.230066, 50.75444, -101.77431, 39.925747, 15.748611, 33.35057, -102.09892, -25.899054, -86.99527, -91.47908, 71.19072, -43.48292, -15.631024, 5

In [43]:
!pip install plotly

# 模型視覺化

In [44]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)